In [27]:
import numpy as np
from astropy.io import fits
from mpdaf.obj import Cube
from astropy import units as u

cube_path = '/Users/mary/Fits image of JWST/ngc6302_ch4-long_s3d.fits'
cube = Cube(cube_path)


pixel_size_arcsec = cube.wcs.get_step(unit=u.arcsec)[0]  
solid_angle = (pixel_size_arcsec * u.arcsec).to(u.rad)**2  
flux_density = cube.data * solid_angle.value * 1e6  
wavelengths = cube.wave.coord() 
range_min, range_max = 124.60, 25.24
mask = (wavelengths >= range_min) & (wavelengths <= range_max)
filtered_wavelengths = wavelengths[mask]
continuum_cube = np.zeros((len(filtered_wavelengths), cube.shape[1], cube.shape[2]))

for y in range(cube.shape[1]):  
    for x in range(cube.shape[2]):  
        pixel_flux = flux_density[:, y, x] 
        continuum_start = pixel_flux[np.argmin(np.abs(wavelengths - range_min))]
        continuum_end = pixel_flux[np.argmin(np.abs(wavelengths - range_max))]
        continuum_line = np.interp(filtered_wavelengths, [range_min, range_max], [continuum_start, continuum_end])
        continuum_cube[:, y, x] = continuum_line

if np.ma.isMaskedArray(continuum_cube):
    continuum_cube = continuum_cube.filled(np.nan) 
output_path = '25continuum_line.fits'
cube_header = fits.getheader(cube_path, ext=1)
cube_header['CRVAL3'] = range_min  # Starting wavelength
cube_header['CDELT3'] = wavelengths[1] - wavelengths[0]  # Wavelength step size
cube_header['NAXIS3'] = len(filtered_wavelengths)  # Number of wavelengths
cube_header['NAXIS1'] = continuum_cube.shape[2]  # X-dimension
cube_header['NAXIS2'] = continuum_cube.shape[1]  # Y-dimension

# Write the FITS file
fits.writeto(output_path, continuum_cube.astype('float32'), header=cube_header, overwrite=True)
print(f"Continuum FITS file saved at {output_path}")



Set OBSGEO-B to    -6.234627 from OBSGEO-[XYZ].
Set OBSGEO-H to 1336593657.111 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-09-10T00:53:49.172' from MJD-AVG.
Set DATE-END to '2023-09-10T03:25:56.696' from MJD-END'. [astropy.wcs.wcs]
/Users/mary/anaconda3/lib/python3.11/site-packages/numpy/lib/function_base.py:1562: UserWarning: Warning: converting a masked element to nan.
  fp = np.asarray(fp)


Continuum FITS file saved at 25continuum_line.fits
